In [1]:
import os.path, time

#implement file chooser
fName = "testCAN.txt"

UnixModTime = os.path.getmtime(fName)
print("Last modified: %s" % time.ctime(os.path.getmtime(fName)))
print("Created: %s" % time.ctime(os.path.getctime(fName)))

with open(fName) as f:
    lines = f.readlines()
    print(f"Read {len(lines)} lines from file {fName}")
    print(lines[0:2])

Last modified: Wed Sep 30 11:09:19 2020
Created: Wed Sep 30 11:09:18 2020
Read 31 lines from file testCAN.txt
['Chn Identifier Flg   DLC  D0...1...2...3...4...5...6..D7       Time     Dir\n', ' 0   217056256 X       8 255 255 255  88  88 255 243 255    4262.442410 R\n']


In [2]:
#TRC HEADER CREATTION
trc_header = """;$FILEVERSION=1.1
;$STARTTIME="""

#SEE https://www.peak-system.com/produktcd/Pdf/English/PEAK_CAN_TRC_File_Format.pdf
#$STARTTIME is the days since 12/30/1899 and the fractional portion of the day since midnight.
#This is slightly incorrect since it is using the end time as the base time.
#NOT CONCERNED
import datetime
fulldatetime = datetime.datetime.strptime(time.ctime(UnixModTime), "%c")
l_date = fulldatetime.date()
f_date = datetime.date(1899, 12, 30)
delta = l_date - f_date
#print(delta.days)

ms = fulldatetime.time()
seconds_per_day = 3600*24
seconds_since_midnight = (fulldatetime - fulldatetime.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
fractional_day = seconds_since_midnight / seconds_per_day
#print(fractional_day)

#WRITE TIMESTAMP
bunkepoch = delta.days + fractional_day
trc_header = trc_header + str(bunkepoch) 

#FINISH BOILERPLATE
trc_header = trc_header + """
;
;   Columns description:
;   ~~~~~~~~~~~~~~~~~~~~~
;   +-Message Number
;   |          +Time offset (ms)
;   |          |        +Type
;   |          |        |        +ID (hex)
;   |          |        |        |     +Data Length Code
;   |          |        |        |     |   +Data bytes (hex)
;   |          |        |        |     |   |
;---+---   ----+----  --+--  ----+---  +  -+ -- -- -- -- -- -- --
"""
print(trc_header)

;$FILEVERSION=1.1
;$STARTTIME=44104.46480324074
;
;   Columns description:
;   ~~~~~~~~~~~~~~~~~~~~~
;   +-Message Number
;   |          +Time offset (ms)
;   |          |        +Type
;   |          |        |        +ID (hex)
;   |          |        |        |     +Data Length Code
;   |          |        |        |     |   +Data bytes (hex)
;   |          |        |        |     |   |
;---+---   ----+----  --+--  ----+---  +  -+ -- -- -- -- -- -- --



In [3]:
#ASC HEADER CREATTION
asc_header = "date "

import datetime
fulldatetime = datetime.datetime.strptime(time.ctime(UnixModTime), "%c")
#date_str = fulldatetime.strftime("%a %b %d %X %Y") #%X time approx doesn't use am/pm
date_str = fulldatetime.strftime("%a %b %d %I:%M:%S %p %Y")
#print(date_str)

asc_header = asc_header + date_str +'\n'
asc_header = asc_header + """base hex  timestamps relative
internal events logged
Begin Triggerblock
"""
print(asc_header)

date Wed Sep 30 11:09:19 AM 2020
base hex  timestamps relative
internal events logged
Begin Triggerblock



In [4]:
from dataclasses import dataclass

#DATA STRUCTURE
#Made this to strongly type parsed data
@dataclass
class CanLine:
    """Class for storing CAN frame."""
    message_number: int
    time: float
    time_offset: float
    transmit: str
    identifier: int
    dlc: int
    data: str
    def __repr__(self):
        return f"Virtual {__class__}"

#Added benefit is __repr__() which allows to easily convert to body of trc file    
@dataclass
class TrcCanLine(CanLine):
    def __repr__(self):
        rep = ''
        SEPARATOR = '  '
        
        COUNTER_JUST =  7
        spaces = COUNTER_JUST - len(str(self.message_number))
        rep = ' '*spaces + str(self.message_number) + ')' + SEPARATOR
        
        TIME_JUST = 10
        time = '%.1f' % self.time_offset
        rep = rep + ' '*(TIME_JUST - len(time)) + time + SEPARATOR
        
        TX_JUST = 5
        rep = rep + self.transmit + ' '*(TX_JUST - len(self.transmit)) + SEPARATOR
        
        ID_JUST = 8
        hexid = '%08X' % self.identifier
        rep = rep + ' '*(ID_JUST - len(hexid)) + hexid + SEPARATOR
        
        rep = rep + str(self.dlc) + SEPARATOR
        
        rep = rep + self.data

        return rep

#Added benefit is __repr__() which allows to easily convert to body of asc file    
@dataclass
class AscCanLine(CanLine):
    def __repr__(self):
        rep = ''
        SEPARATOR = '  '
        
        TIME_JUST = 11
        time = '%08f' % (self.time_offset / 1000)
        rep = rep +' '*(TIME_JUST - len(time)) + time + ' 1'
        
        
        hexid = '%08X' % self.identifier
        rep = rep + SEPARATOR + hexid + 'x'
        
        rep = rep + ' '*7 + self.transmit
        
        rep = rep + SEPARATOR + ' d ' + str(self.dlc) + ' ' + self.data.rstrip()
                
        return rep

In [5]:
#CAN PARSER
#This will need to be modified on most runs
#Depending on how CAN KING is configured will have to skip lines or convert from base 10

#extremely inefficient; whatever.
asc_canlines = []
trc_canlines = []

prevtime = None
line_count = 1

#CAN KING had 2 formatters so lines were duplicated.  Only grab odd lines.
for i in range(1, len(lines) - 1, 2):
    line = lines[i]

#for line in lines[1:-2]:
    splitline = line.split()
    
    timestamp = float(splitline[-2])
    
    if prevtime is not None:
        time_offset = (timestamp - prevtime) * 1000
    else:
        time_offset = 0.0
    prevtime = timestamp
    
    
    if splitline[-1] == 'R':
        transmit = 'Rx'
    else:
        transmit = 'Tx'
        
    dlc = int(splitline[3])
    
    data = ''
    if dlc > 0 and dlc < 0xFF:
        for i in range(4, 4+dlc):
            data = data + '%02X' % int(splitline[i]) + ' '
    
    asc_canlines.append(AscCanLine(line_count, timestamp, time_offset, transmit, int(splitline[1]), dlc, data))
    trc_canlines.append(TrcCanLine(line_count, timestamp, time_offset, transmit, int(splitline[1]), dlc, data))
    line_count = line_count + 1

print(asc_canlines[-3:-1])
print(trc_canlines[-3:-1])

[   0.000520 1  18FF0E00x       Rx   d 8 FC 03 92 04 E8 03 E8 03,    0.000590 1  18FF2700x       Rx   d 8 02 00 00 02 FF FF FF FF]
[     13)         0.5  Rx     18FF0E00  8  FC 03 92 04 E8 03 E8 03 ,      14)         0.6  Rx     18FF2700  8  02 00 00 02 FF FF FF FF ]


In [6]:
#Write the trace file
oName = fName.split('.')[0] + '.trc'
with open(oName, "wt") as of:
    of.write(trc_header)
    for i in trc_canlines:
        of.write(i.__repr__() + '\n')
print(f"Wrote {os.path.getsize(oName)} bytes to {oName}")

Wrote 1490 bytes to testCAN.trc


In [7]:
#Write the asc file
oName = fName.split('.')[0] + '.asc'
with open(oName, "wt") as of:
    of.write(asc_header)
    for i in asc_canlines:
        of.write(i.__repr__() + '\n')
    of.write('End Triggerblock\n')
print(f"Wrote {os.path.getsize(oName)} bytes to {oName}")

Wrote 1102 bytes to testCAN.asc
